# RVC v2 Disconnected



In [ ]:
# Colab'ın antrenman sırasında "hareketsizlik" nedeniyle sizi zaman aşımına uğratmasını önlemek için muhtemelen bir bağlantı kesme önleme yöntemi isteyeceksiniz.
# Burada bir tane vermeyeceğim, ancak bunları bulmak zor değil. Colab AI topluluğunda arama yaparsanız bir tane bulabilirsiniz ;)

# Gereklilikler
İlk bunu çalıştır!


In [ ]:
#@title Install Dependencies
import subprocess

packages = ['build-essential', 'python3-dev', 'ffmpeg', 'aria2']
pip_packages = ['pip', 'setuptools', 'wheel', 'httpx==0.23.0', 'faiss-gpu', 'fairseq', 'ffmpeg', 'ffmpeg-python', 'praat-parselmouth', 'pyworld', 'numpy==1.23.5', 'numba==0.56.4', 'librosa==0.9.2', 'gdown', 'onnxruntime']
print("Updating and installing system packages...")
for package in packages:
  print(f"Installing {package}...")
  subprocess.check_call(['apt-get', 'install', '-qq', '-y', package])

print("Updating and installing pip packages...")
subprocess.check_call(['pip', 'install', '--upgrade'] + pip_packages)

print('Packages up to date.')
firsttry = True

Updating and installing system packages...
Installing build-essential...
Installing python3-dev...
Installing ffmpeg...
Installing aria2...
Updating and installing pip packages...
Packages up to date.


In [ ]:
#@title Clone Repositories
import os

# READ ME BEFORE CHANGING THINGS
# If you're attempting to replace the imports here with Applio-RVC, it will not work due to requirement discrepancies across the entire notebook.
# I will not be porting this notebook to Applio due to the failure of the Applio team to provide backwards compatibility with the Crepe and Mangio-Crepe f0 feature format.
# DO NOT ASK. IT WILL NOT HAPPEN.

os.chdir('/content/')
#Credit to Redoverflow on the AI Hub Discord for (indirectly) suggesting this variant of Mangio RVC to me.
!git clone -b pr-optimization --single-branch https://github.com/alexlnkp/Mangio-RVC-Tweaks.git
#Rename to keep backwards compatibility with old variants of Disconnected
os.rename("/content/Mangio-RVC-Tweaks", "/content/Mangio-RVC-Fork")
!git clone https://github.com/maxrmorrison/torchcrepe.git
!mv torchcrepe/torchcrepe Mangio-RVC-Fork/
!rm -rf torchcrepe  # Delete the torchcrepe repository folder
os.chdir('/content/Mangio-RVC-Fork')

now_dir = "/content/Mangio-RVC-Fork"
os.makedirs(os.path.join(now_dir, "logs"), exist_ok=True)
os.makedirs(os.path.join(now_dir, "weights"), exist_ok=True)


Cloning into 'Mangio-RVC-Tweaks'...
remote: Enumerating objects: 3102, done.
remote: Counting objects: 100% (1092/1092), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 3102 (delta 1026), reused 992 (delta 992), pack-reused 2010
Receiving objects: 100% (3102/3102), 13.75 MiB | 26.17 MiB/s, done.
Resolving deltas: 100% (1974/1974), done.
Cloning into 'torchcrepe'...
remote: Enumerating objects: 442, done.
remote: Counting objects: 100% (437/437), done.
remote: Compressing objects: 100% (194/194), done.
remote: Total 442 (delta 253), reused 395 (delta 228), pack-reused 5
Receiving objects: 100% (442/442), 72.20 MiB | 26.80 MiB/s, done.
Resolving deltas: 100% (253/253), done.
Updating files: 100% (27/27), done.


In [ ]:
#@title GPU Check
import torch

ngpu = torch.cuda.device_count()
gpu_infos = []
mem = []
if_gpu_ok = False

if torch.cuda.is_available() or ngpu != 0:
  for i in range(ngpu):
    gpu_name = torch.cuda.get_device_name(i)
    if any(
        value in gpu_name.upper()
        for value in ["10", "16", "20", "30", "40", "A2", "A3", "A4", "P4", "A50", "500", "A60", "70", "80", "90", "M4", "T4", "TITAN"]
    ):
      if_gpu_ok = True
      print("Compatible GPU detected: %s" % gpu_name)
      gpu_infos.append("%s\t%s" % (i, gpu_name))
      mem.append(int(torch.cuda.get_device_properties(i).total_memory / 1024 / 1024 / 1024 + 0.4))

if if_gpu_ok and len(gpu_infos) > 0:
  gpu_info = "\n".join(gpu_infos)

else:
  raise Exception("No GPU detected; training cannot continue. Please change your runtime type to a GPU.")
gpus = "-".join(i[0] for i in gpu_infos)

Compatible GPU detected: Tesla T4


In [ ]:
#@title Mount Drive
from google.colab import drive
if not os.path.exists('/content/drive'):
  drive.mount('/content/drive')
else:
  print('Drive is already mounted. Proceed.')

os.makedirs('/content/drive/MyDrive/rvcDisconnected', exist_ok=True)

Mounted at /content/drive


In [ ]:
#@title Download Pretrained Models
#Didn't ask.

#V1 Models
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G32k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D32k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G40k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D40k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0G48k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/v1/f0D48k.pth -d /content/Mangio-RVC-Fork/pretrained -o f0D48k.pth

#V2 Models
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G32k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D32k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D32k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G40k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D40k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D40k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0G48k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0G48k.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/f0D48k.pth -d /content/Mangio-RVC-Fork/pretrained_v2 -o f0D48k.pth

#RMVPE and Hubert
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/hubert_base.pt -d /content/Mangio-RVC-Fork -o hubert_base.pt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/rmvpe.pt -d /content/Mangio-RVC-Fork -o rmvpe.pt

#fp_16 Variant JSONs
!rm -rf /content/Mangio-RVC-Fork/configs/32k.json
!rm -rf /content/Mangio-RVC-Fork/configs/40k.json
!rm -rf /content/Mangio-RVC-Fork/configs/48k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/32k.json -d /content/Mangio-RVC-Fork/configs -o 32k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/40k.json -d /content/Mangio-RVC-Fork/configs -o 40k.json
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/Kit-Lemonfoot/RVC_DidntAsk/resolve/main/48k.json -d /content/Mangio-RVC-Fork/configs -o 48k.json


Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
8b033a|OK  |   186MiB/s|/content/Mangio-RVC-Fork/pretrained/f0G32k.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
e16061|OK  |   233MiB/s|/content/Mangio-RVC-Fork/pretrained/f0D32k.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
cc640f|OK  |   225MiB/s|/content/Mangio-RVC-Fork/pretrained/f0G40k.pth

Status Legend:
(OK):download completed.

Download Results:
gid   |stat|avg speed  |path/URI
======+====+===========+=======================================================
318919|OK  |   277MiB/s|/content/Mangio-RVC-Fork/pretrained/f0D40k.pth

Status Legend:
(OK):download completed.

Download Results:
gid  

In [ ]:
#@title Setup CSVDB
#...Alright, you made your point.
import csv

if not os.path.isdir("csvdb/"):
  os.makedirs("csvdb")
  frmnt, stp = open("csvdb/formanting.csv", "w", newline=""), open("csvdb/stop.csv", "w", newline="")
  csv_writer = csv.writer(frmnt, delimiter=",")
  csv_writer.writerow([False, 1.0, 1.0])
  csv_writer = csv.writer(stp, delimiter=",")
  csv_writer.writerow([False])
  frmnt.close()
  stp.close()

global DoFormant, Quefrency, Timbre
DoFormant, Quefrency, Timbre = False, 1.0, 1.0

# Eğitim Değişkenlerini Ayarla

In [ ]:
#@title Eğitim Değişkenlerini Ayarla
now_dir = "/content/Mangio-RVC-Fork"
experiment_name = "yvonne" #@param {type:"string"}
path_to_training_folder = "/content/dataset/"
model_architecture = "v2" #@param ["v1","v2"] {allow-input: false}
target_sample_rate = "48k" #@param ["32k", "40k", "48k"] {allow-input: false}
cpu_threads = 2 #@param {type:"integer"}
speaker_id = 0 #@param {type:"integer"}
pitch_extraction_algorithm = "rmvpe" #@param ["harvest", "crepe", "mangio-crepe", "rmvpe"] {allow-input: false}
crepe_hop_length = 64 #@param {type:"integer"}
pitch_guidance = True #@param {type:"boolean"}

assert crepe_hop_length!=None, "You need to input something for crepe_hop_length, silly."
assert crepe_hop_length>0, "Hop length must be more than 0."
assert crepe_hop_length<=512, "Save frequency must be less than 512."

if(experiment_name == "experiment_name"):
  print("Warning: Your experiment name should be changed to the name of your dataset.")

#ön işlemler
Bu hücreleri model başına yalnızca bir kez çalıştırmanız gerekir.

In [ ]:
#@title Dataseti Yükle
#@markdown Henüz mevcut değilse, Google Drive'ınızda 'rvcDisconnected' adında bir klasör oluşturun ve zip dosyanızı buraya yerleştirin. Bu, 'rvcDisconnected' klasörünün içinde aşağıdaki ZIP dosyasını arayacaktır.
dataset = "dataset.zip"  #@param {type:"string"}

#@markdown Bu yükleyici veri kümelerini So-Vits-SVC veri kümesi yükleyicisine benzer bir şekilde yükleyecektir. En iyi sonuçlar için, veri kümeniz bu şekilde biçimlendirilmelidir:
#@markdown ```
#@markdown zipfile.zip
#@markdown └───character_name
#@markdown     ├───file1.wav
#@markdown     ├───...
#@markdown     └───file999.wav
#@markdown ```
#@markdown Ses dosya adları önemli değildir. En iyi uyumluluk için tüm ses dosyaları WAV formatında olmalıdır.

# TODO: Add something to convert non-WAVs to WAV

import os
import shutil

directories=[]

def sanitize_directory(directory):
  for filename in os.listdir(directory):
    file_path = os.path.join(directory, filename)
    if os.path.isfile(file_path):
      if filename == ".DS_Store" or filename.startswith("._") or not filename.endswith(('.wav', '.flac', '.mp3', '.ogg', '.m4a')):
        os.remove(file_path)
    elif os.path.isdir(file_path):
      #Get rid of the MACOSX directory just so it doesn't mess with renaming later
      if(filename == "__MACOSX"):
        shutil.rmtree(file_path)
        continue
      #Append the directory to directories for future dataset check, then recurse.
      directories.append(file_path)
      sanitize_directory(file_path)

dataset_path = '/content/drive/MyDrive/rvcDisconnected/' + dataset
final_directory = '/content/dataset'
temp_directory = '/content/temp_dataset'

if os.path.exists(final_directory):
  print("Dataset folder already found. Wiping...")
  shutil.rmtree(final_directory)
if os.path.exists(temp_directory):
  print("Temporary folder already found. Wiping...")
  shutil.rmtree(temp_directory)

if not os.path.exists(dataset_path):
  raise Exception(f'I can\'t find {dataset} in {os.path.dirname(dataset_path)}.')

os.makedirs(final_directory, exist_ok=True)
os.makedirs(temp_directory, exist_ok=True)
#Oops.
!unzip -d "{temp_directory}" -B "{dataset_path}"
print("Sanitizing...")
sanitize_directory(temp_directory)

if(len(directories) == 0):
  #If there's no directories, we're dealing with a ZIP of just audio files.
  #Move everything to /dataset/experiment_name/.
  print("Dataset Type: Audio Files (Single Speaker)")
  expDir=os.path.join(final_directory, experiment_name)
  os.makedirs(expDir, exist_ok=True)
  for r, _, f in os.walk(temp_directory):
    for name in f:
      !cp "{temp_directory}/{name}" "{expDir}"
elif(len(directories) == 1):
  #If there's only one directory, we're dealing with a single speaker.
  #Rename the folder to experiment_name and move it to /dataset/.
  print("Dataset Type: Single Speaker")
  fi = os.path.join(temp_directory, experiment_name)
  os.rename(directories[0], fi)
  shutil.move(fi, final_directory)

else:
  #If anything else, we're dealing with multispeaker.
  #Move all folders to /dataset/ indiscriminately.
  print("Dataset Type: Multispeaker")
  for fi in directories:
    shutil.move(fi, final_directory)

shutil.rmtree(temp_directory)

print("Dataset imported.")


Archive:  /content/drive/MyDrive/rvcDisconnected/dataset.zip
  inflating: /content/temp_dataset/yvonne_0.wav  
  inflating: /content/temp_dataset/yvonne_1.wav  
  inflating: /content/temp_dataset/yvonne_10.wav  
  inflating: /content/temp_dataset/yvonne_100.wav  
  inflating: /content/temp_dataset/yvonne_101.wav  
  inflating: /content/temp_dataset/yvonne_102.wav  
  inflating: /content/temp_dataset/yvonne_103.wav  
  inflating: /content/temp_dataset/yvonne_104.wav  
  inflating: /content/temp_dataset/yvonne_105.wav  
  inflating: /content/temp_dataset/yvonne_106.wav  
  inflating: /content/temp_dataset/yvonne_107.wav  
  inflating: /content/temp_dataset/yvonne_108.wav  
  inflating: /content/temp_dataset/yvonne_109.wav  
  inflating: /content/temp_dataset/yvonne_11.wav  
  inflating: /content/temp_dataset/yvonne_110.wav  
  inflating: /content/temp_dataset/yvonne_111.wav  
  inflating: /content/temp_dataset/yvonne_112.wav  
  inflating: /content/temp_dataset/yvonne_113.wav  
  inflati

In [ ]:
#@title Ön işleme
# Change the Experiment Name and the Path to Training Folder. You shouldn't need to change anything else.

import os
import subprocess

assert cpu_threads>0, "CPU threads not allocated correctly."

sr = int(target_sample_rate.rstrip('k'))*1000
pttf = path_to_training_folder + experiment_name
os.makedirs("%s/logs/%s" % (now_dir, experiment_name), exist_ok=True)

cmd = "python trainset_preprocess_pipeline_print.py \"%s\" %s %s \"%s/logs/%s\" 1" % (pttf, sr, cpu_threads, now_dir, experiment_name)
print(cmd)
!$cmd

python trainset_preprocess_pipeline_print.py "/content/dataset/yvonne" 48000 2 "/content/Mangio-RVC-Fork/logs/yvonne" 1
start preprocess
['trainset_preprocess_pipeline_print.py', '/content/dataset/yvonne', '48000', '2', '/content/Mangio-RVC-Fork/logs/yvonne', '1']
thread:0:   0% 0/87 [00:00<?, ?it/s]
thread:0:   1% 1/87 [00:02<04:02,  2.82s/it]
thread:0:   2% 2/87 [00:03<02:32,  1.80s/it]
thread:1:   2% 2/86 [00:03<02:31,  1.81s/it]
thread:0:   3% 3/87 [00:04<01:52,  1.33s/it]
thread:0:   5% 4/87 [00:05<01:28,  1.07s/it]
thread:0:   6% 5/87 [00:06<01:17,  1.06it/s]
thread:0:   8% 7/87 [00:07<01:02,  1.28it/s]
thread:0:   9% 8/87 [00:08<00:59,  1.33it/s]
thread:0:  10% 9/87 [00:08<00:56,  1.37it/s]
thread:0:  11% 10/87 [00:09<00:52,  1.46it/s]
thread:0:  13% 11/87 [00:09<00:51,  1.47it/s]
thread:0:  14% 12/87 [00:10<00:50,  1.49it/s]
thread:0:  15% 13/87 [00:11<00:49,  1.49it/s]
thread:0:  16% 14/87 [00:11<00:48,  1.51it/s]
thread:0:  17% 15/87 [00:12<00:46,  1.55it/s]
thread:0:  18% 16

In [ ]:
#@title Feature Extraction

#pitch_extraction_algorithm = "harvest" #@param ["harvest", "crepe", "mangio-crepe"] {allow-input: false}
#crepe_hop_length = 128 #@param {type:"slider", min:1, max:512, step:1}
#pitch_guidance = True #@param {type:"boolean"}

gpuList = gpus.split("-")
cmd = "python extract_f0_print.py \"%s/logs/%s\" %s %s %s" % (now_dir, experiment_name, cpu_threads, pitch_extraction_algorithm, crepe_hop_length)
print(cmd)
!$cmd

leng = len(gpus)

cmd = "python extract_feature_print.py %s %s %s %s \"%s/logs/%s\" %s" % ("device", leng, 0, 0, now_dir, experiment_name, model_architecture)
print(cmd)
!$cmd


python extract_f0_print.py "/content/Mangio-RVC-Fork/logs/yvonne" 2 rmvpe 64
['extract_f0_print.py', '/content/Mangio-RVC-Fork/logs/yvonne', '2', 'rmvpe', '64']
Using f0 method: rmvpe
thread:0, f0ing, Hop-Length:64:   0% 0/380 [00:00<?, ?it/s]
  0% 0/379 [00:00<?, ?it/s]
thread:1, f0ing, Hop-Length:64:   0% 0/379 [00:00<?, ?it/s]
thread:0, f0ing, Hop-Length:64:   0% 1/380 [00:09<59:02,  9.35s/it]
thread:0, f0ing, Hop-Length:64:   1% 2/380 [00:09<25:25,  4.04s/it]
thread:0, f0ing, Hop-Length:64:   1% 3/380 [00:09<14:27,  2.30s/it]
thread:0, f0ing, Hop-Length:64:   1% 4/380 [00:10<09:10,  1.46s/it]
thread:0, f0ing, Hop-Length:64:   1% 5/380 [00:10<06:28,  1.04s/it]
thread:0, f0ing, Hop-Length:64:   2% 6/380 [00:10<04:47,  1.30it/s]
thread:0, f0ing, Hop-Length:64:   2% 7/380 [00:10<03:35,  1.73it/s]
thread:0, f0ing, Hop-Length:64:   2% 8/380 [00:11<02:54,  2.14it/s]
thread:0, f0ing, Hop-Length:64:   2% 9/380 [00:11<02:20,  2.64it/s]
thread:0, f0ing, Hop-Length:64:   3% 10/380 [00:11<02:05

In [ ]:
#@title Önceden işlenmiş Dataset dosyalarını Google Drive'a kaydedin
#Compress dataset folder
loc = "%s/logs/%s" % (now_dir, experiment_name)
!zip -r rvcLogs.zip "{loc}"
DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
!mkdir -p "{DATASET_PATH_DRIVE}"
!cp /content/Mangio-RVC-Fork/rvcLogs.zip "{DATASET_PATH_DRIVE}"

  adding: content/Mangio-RVC-Fork/logs/yvonne/ (stored 0%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/ (stored 0%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/58_5.wav.npy (deflated 17%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/101_2.wav.npy (deflated 20%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/171_3.wav.npy (deflated 32%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/62_1.wav.npy (deflated 33%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/51_4.wav.npy (deflated 42%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/15_4.wav.npy (deflated 46%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/45_4.wav.npy (deflated 22%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/33_5.wav.npy (deflated 32%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/150_3.wav.npy (deflated 10%)
  adding: content/Mangio-RVC-Fork/logs/yvonne/2b-f0nsf/97_3.wav.npy (deflated 50%)
  adding: content/Mangio-RVC-Fork/logs/y

# Eğitim
Ayrıca devam ettirme kodunu da içerir.

In [ ]:
#@title Önceden işlenmiş Dataset dosyalarını Google Drive'dan yükleme (devam ettirmek için)
#@markdown Google Drive'da önceden işlenmiş Dataset dosyalarınız varsa, ön işleme adımlarını yeniden çalıştırmak yerine bunları buraya yükleyebilirsiniz
import os

BACK_UP_DATASET_PATH = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name

#Prevent people from loading the ZIP over existing files
ok=True
if(os.path.exists("/content/Mangio-RVC-Fork/logs/"+experiment_name+"/2a_f0")):
  print("Dataset files already loaded, skipping.")
  ok=False

if ok:
  !unzip "{BACK_UP_DATASET_PATH}/rvcLogs.zip" -d /

Dataset files already loaded, skipping.


In [ ]:
#@title Modeli Drive Dan Colab'a Aktarma(Devam Ettirmek için)
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
#@markdown Tamamlanmamış modeller için adım sayınızı ve epoch sayınızı manuel olarak girmek istiyorsanız bu seçenekleri kullanın. Eğitiminiz bittiyse bu seçeneği kullanmayın.
STEPCOUNT = 000 #@param {type:"integer"}

print("Copying model files...")
!cp "{DATASET_PATH_DRIVE}/D_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/G_{STEPCOUNT}.pth" "{DATASET_PATH_COLAB}"
!cp "{DATASET_PATH_DRIVE}/config.json" "{DATASET_PATH_COLAB}"

print("Copying Tensorboard TFEVENT files...")
for r, _, f in os.walk(DATASET_PATH_DRIVE):
  for name in f:
    if(name.startswith("events.out.tfevents")):
      !cp "{DATASET_PATH_DRIVE}/{name}" "{DATASET_PATH_COLAB}"

print("All done. Welcome back!")

Copying model files...
cp: cannot stat '/content/drive/MyDrive/rvcDisconnected/yvonne/D_0.pth': No such file or directory
cp: cannot stat '/content/drive/MyDrive/rvcDisconnected/yvonne/G_0.pth': No such file or directory
cp: cannot stat '/content/drive/MyDrive/rvcDisconnected/yvonne/config.json': No such file or directory
Copying Tensorboard TFEVENT files...
All done. Welcome back!


In [ ]:
import os
import math
from random import shuffle

# I will not be adding an autosave feature. Do not ask.

#@title Eğitim
save_frequency = 50 #@param {type:"integer"}
total_epochs = 200 #@param {type:"integer"}
batch_size = 8 #@param {type:"integer"}

save_only_latest_ckpt = False #@param {type:"boolean"}
cache_all_training_sets = False #@param {type:"boolean"}
save_small_final_model = True #@param {type:"boolean"}
#@markdown Otomatik olarak hesaplanan günlük aralığının çok hatalı olduğu bilinmektedir ve bir epok bitirme ile Tensorboard yazmaları arasında gecikmelere neden olabilir. İsterseniz, burada manuel olarak bir günlük aralığı tanımlayabilirsiniz.
use_manual_stepToEpoch = False #@param {type:"boolean"}
manual_stepToEpoch = 000 #@param {type:"integer"}

assert save_frequency!=None, "You need to input something for save_frequency, silly."
assert save_frequency>0, "Save frequency must be more than 0."
if(save_frequency>50):print("...A save frequency of %s? A bit high, but... alright then."%save_frequency)
assert total_epochs!=None, "You need to input something for total_epochs, silly."
assert total_epochs>0, "Total epochs must be more than 0."
if(total_epochs>10000):print("...A total epoch count of of %s? This is going to overtrain, but... alright then."%total_epochs)
assert batch_size!=None, "You need to input something for batch_size, silly."
assert batch_size>0, "Batch size must be more than 0."
assert batch_size<=40, "Batch size must be less than 40. (I'd reccomend a value between 6 and 12 for Colab.)"

pretrained_base = "pretrained/" if model_architecture == "v1" else "pretrained_v2/"
exp_dir = "%s/logs/%s" % (now_dir, experiment_name)

pretrainedD = "%sf0D%s.pth" % (pretrained_base, target_sample_rate)
pretrainedG = "%sf0G%s.pth" % (pretrained_base, target_sample_rate)

#Log interval
log_interval = 1
liFolderPath = os.path.join(exp_dir, "1_16k_wavs")
if(os.path.exists(liFolderPath) and os.path.isdir(liFolderPath)):
  wav_files = [f for f in os.listdir(liFolderPath) if f.endswith(".wav")]
  if wav_files:
    sample_size = len(wav_files)
    log_interval = math.ceil(sample_size / batch_size)
    if log_interval > 1:
      log_interval += 1

if log_interval > 250 and not use_manual_stepToEpoch:
  print("That's a big dataset you got there. Log interval normalized to 200 steps from %s steps." % log_interval)
  log_interval = 200

if use_manual_stepToEpoch:
  log_interval = manual_stepToEpoch

#Create Python command
cmd = "python train_nsf_sim_cache_sid_load_pretrain.py -e \"%s\" -sr %s -f0 %s -bs %s -g %s -te %s -se %s %s %s -l %s -c %s -sw %s -v %s -li %s" % (
    experiment_name,
    target_sample_rate,
    1,
    batch_size,
    0,
    total_epochs,
    save_frequency,
    "-pg %s" % pretrainedG if pretrainedG != "" else "\b",
    "-pd %s" % pretrainedD if pretrainedD != "" else "\b",
    1 if save_only_latest_ckpt else 0,
    1 if cache_all_training_sets else 0,
    1 if save_small_final_model else 0,
    model_architecture,
    log_interval,
)
print(cmd)

#Create mute filelist
exp_dir = "%s/logs/%s" % (now_dir, experiment_name)
gt_wavs_dir = "%s/0_gt_wavs" % (exp_dir)
feature_dir = (
  "%s/3_feature256" % (exp_dir)
  if model_architecture == "v1"
  else "%s/3_feature768" % (exp_dir)
)
f0_dir = "%s/2a_f0" % (exp_dir)
f0nsf_dir = "%s/2b-f0nsf" % (exp_dir)
names = (
  set([name.split(".")[0] for name in os.listdir(gt_wavs_dir)])
  & set([name.split(".")[0] for name in os.listdir(feature_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0_dir)])
  & set([name.split(".")[0] for name in os.listdir(f0nsf_dir)])
)
opt = []
for name in names:
  opt.append(
    "%s/%s.wav|%s/%s.npy|%s/%s.wav.npy|%s/%s.wav.npy|%s"
    % (
      gt_wavs_dir.replace("\\", "\\\\"),
      name,
      feature_dir.replace("\\", "\\\\"),
      name,
      f0_dir.replace("\\", "\\\\"),
      name,
      f0nsf_dir.replace("\\", "\\\\"),
      name,
      speaker_id,
    )
  )
fea_dim = 256 if model_architecture == "v1" else 768
for _ in range(2):
  opt.append(
      "%s/logs/mute/0_gt_wavs/mute%s.wav|%s/logs/mute/3_feature%s/mute.npy|%s/logs/mute/2a_f0/mute.wav.npy|%s/logs/mute/2b-f0nsf/mute.wav.npy|%s"
      % (now_dir, target_sample_rate, now_dir, fea_dim, now_dir, now_dir, speaker_id)
  )
shuffle(opt)
with open("%s/filelist.txt" % exp_dir, "w") as f:
  f.write("\n".join(opt))
print("Mute filelist written. Best of luck training!")


%cd /content/Mangio-RVC-Fork
%load_ext tensorboard
%tensorboard --logdir /content/Mangio-RVC-Fork/logs

os.chdir('/content/Mangio-RVC-Fork')
!$cmd


NameError: name 'model_architecture' is not defined

In [ ]:
#@title Model dosyasını Colab'dan Drive'a aktarma
import os

DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  !mkdir -p "{DATASET_PATH_DRIVE}"

#@markdown Yalnızca weights üzerinden kopyalama yapmak istiyorsanız bu seçeneği kullanın.
skip_models = False #@param {type:"boolean"}
#@markdown Tamamlanmamış modeller için adım sayınızı ve epoch sayınızı manuel olarak girmek istiyorsanız bu seçenekleri kullanın. Eğitiminiz bittiyse bu seçeneği kullanmayın.
manual_save = False #@param {type:"boolean"}
STEPCOUNT = 000 #@param {type:"integer"}
EPOCHCOUNT = 000 #@param {type:"integer"}

finished=False
potential="/content/Mangio-RVC-Fork/weights/"+experiment_name+".pth"
if os.path.exists(potential):
  finished = True

#VERY hacky. Might break stuff, report to me if it does.
print("Detecting latest model...")
if(not manual_save):
  currentMax = 0
  for r, _, f in os.walk("/content/Mangio-RVC-Fork/weights/"):
    for name in f:
      if(name.endswith(".pth") and (name!=experiment_name+".pth")):
        #Check to see if this PTH is what we're looking for.
        if(name.find(experiment_name)==-1):
          continue
        #Determine Epochcount+Stepcount Phase 1
        pot = name.split('_')
        ep=pot[len(pot)-2][1:]
        #If what we got from the epoch count section of the filename isn't a number, multiple completed models are in weights.
        #Skip it if that happens.
        if(not ep.isdecimal()):
          continue
        #Determine Epochcount+Stepcount Phase 2
        ep=int(ep)
        if ep>currentMax:
          currentMax=ep
          step=pot[len(pot)-1].split('.')
          step=int(step[0][1:])
          EPOCHCOUNT=ep
          STEPCOUNT=step

TSTEP = STEPCOUNT
if(not skip_models):
  print("Copying model files...")
  if(save_only_latest_ckpt):
    TSTEP=2333333
  !cp "{DATASET_PATH_COLAB}/D_{TSTEP}.pth" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/G_{TSTEP}.pth" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/config.json" "{DATASET_PATH_DRIVE}"

print("Copying Tensorboard TFEVENT files...")
for r, d, f in os.walk(DATASET_PATH_COLAB):
  for name in f:
    if(name.startswith("events.out.tfevents") and os.path.exists(os.path.join(DATASET_PATH_COLAB, name))):
      !cp "{DATASET_PATH_COLAB}/{name}" "{DATASET_PATH_DRIVE}"

print("Copying weight file...")
if(finished):
  !cp "{potential}" "{DATASET_PATH_DRIVE}"
else:
  !cp "/content/Mangio-RVC-Fork/weights/{experiment_name}_e{EPOCHCOUNT}_s{STEPCOUNT}.pth" "{DATASET_PATH_DRIVE}"

print("All done!")

In [ ]:
#@title Index oluşturma
#@markdown Bu hücreyi çalıştırmadan önce Feature Extractin işleminin başarıyla çalıştığından emin olun.

#@markdown **Not: Nadiren de olsa bu hücrede çalışma zamanının yeniden başlatılmasını gerektiren bir hatayla karşılaşabilirsiniz.**
#@markdown **Böyle bir durumda çalışma zamanını yeniden başlatın, "Eğitim Değişkenlerini Ayarla" hücresini yeniden çalıştırın ve ardından bu hücreyi yeniden çalıştırın.**

#@markdown Dizin eğitimi tarafından oluşturulan iki ekstra dosyayı Google Drive'ınıza kaydetmek istiyorsanız bu seçeneği kullanın. (Normalde yalnızca eklenen dizin gereklidir.)
save_extra_files_to_drive = False #@param {type:"boolean"}

#Oh dear lord why is this baked into infer-web I hate this
import os
import sys
import traceback
import numpy as np
import faiss

#from sklearn.cluster import MiniBatchKMeans

exp_dir = "%s/logs/%s" % (now_dir, experiment_name)
os.makedirs(exp_dir, exist_ok=True)
feature_dir = (
    "%s/3_feature256" % (exp_dir)
    if model_architecture == "v1"
    else "%s/3_feature768" % (exp_dir)
)
print(feature_dir)
if not os.path.exists(feature_dir):
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")
listdir_res = list(os.listdir(feature_dir))
if len(listdir_res) == 0:
  raise Exception("No features exist for this model yet. Did you run Feature Extraction?")

try:
  from sklearn.cluster import MiniBatchKMeans
except:
  print("Due to a bug with Colab, we will need to reinstall Numpy real quick. Give me a sec!")
  !pip install -U numpy
  print("Numpy reinstalled. Please restart the runtime, and then re-run the \"Set Training Variables\" cell to continue.")
  sys.exit()
else:
  print("Proper Numpy version detected.")

infos=[]
npys=[]
for name in sorted(listdir_res):
  phone = np.load("%s/%s" % (feature_dir, name))
  npys.append(phone)
big_npy = np.concatenate(npys, 0)
big_npy_idx = np.arange(big_npy.shape[0])
np.random.shuffle(big_npy_idx)
if big_npy.shape[0] > 2e5:
  print("Trying doing kmeans %s shape to 10k centers." % big_npy.shape[0])
  try:
    big_npy = (
        MiniBatchKMeans(
            n_clusters=10000,
            verbose=True,
            batch_size=256,
            compute_labels = False,
            init="random"
        )
        .fit(big_npy)
        .cluster_centers_

    )
  except:
    info = traceback.format_exc()
    print(info)

np.save("%s/total_fea.npy" % exp_dir, big_npy)
n_ivf = min(int(16*np.sqrt(big_npy.shape[0])), big_npy.shape[0] // 39)
print("%s,%s" % (big_npy.shape, n_ivf))
index = faiss.index_factory(256 if model_architecture == "v1" else 768, "IVF%s,Flat" % n_ivf)
print("Training index...")
index_ivf = faiss.extract_index_ivf(index)
index_ivf.nprobe = 1
index.train(big_npy)
faiss.write_index(
    index,
    "%s/trained_IVF%s_Flat_nprobe_%s_%s_%s.index" % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)
print("Adding...")
batch_size_add = 8192
for i in range(0, big_npy.shape[0], batch_size_add):
  index.add(big_npy[i:i+batch_size_add])
faiss.write_index(
    index,
    "%s/added_IVF%s_Flat_nprobe_%s_%s_%s.index"
    % (exp_dir, n_ivf, index_ivf.nprobe, experiment_name, model_architecture)
)

npr = index_ivf.nprobe

print("Saving files to Drive...")
DATASET_PATH_DRIVE = "/content/drive/MyDrive/rvcDisconnected/" + experiment_name
if(not os.path.exists(DATASET_PATH_DRIVE)):
  !mkdir -p "{DATASET_PATH_DRIVE}"
DATASET_PATH_COLAB = "/content/Mangio-RVC-Fork/logs/" + experiment_name
if(save_extra_files_to_drive):
  !cp "{DATASET_PATH_COLAB}/total_fea.npy" "{DATASET_PATH_DRIVE}"
  !cp "{DATASET_PATH_COLAB}/trained_IVF{n_ivf}_Flat_nprobe_{npr}_{experiment_name}_{model_architecture}.index" "{DATASET_PATH_DRIVE}"
!cp "{DATASET_PATH_COLAB}/added_IVF{n_ivf}_Flat_nprobe_{npr}_{experiment_name}_{model_architecture}.index" "{DATASET_PATH_DRIVE}"

print("All done! Your index file has completed training.")
try:
  firsttry
except:
  print("If you had to restart the runtime, disconnect and delete the runtime in order to continue. (Restarting the runtime again will not work.)")



# TODO, Changelogs, Credits, and Special Thanks

Bu Colab Ai Lab Tarafından Türkçeye çevrilmiştir

**Bu Colab Bu kişiler Tarafından Yazıldı [Kit Lemonfoot](https://huggingface.co/Kit-Lemonfoot) / [Noel Shirogane's High Flying Birds](https://www.youtube.com/@NoelShiroganesHighFlyingBirds)**



*Based on the work of the [RVC Project](https://github.com/RVC-Project), [Mangio261](https://github.com/Mangio621/), [Kalomaze](https://github.com/kalomaze), [Alexlnkp](https://github.com/alexlnkp), the [Pony Preservation Project](https://boards.4channel.org/mlp/catalog#s=Pony%20Preservation%20Project), and many others in the RVC / voice AI community* ❤

**Credits**
*   [Kit Lemonfoot](https://huggingface.co/Kit-Lemonfoot) - writing this notebook
*   [RVC Project](https://github.com/RVC-Project) - Created RVC, obviously
*   [LJ1995](https://huggingface.co/lj1995) - Pretrained RVC models
*   [Mangio261](https://github.com/Mangio621/) - Creating the Mangio RVC fork
*   [Kalomaze](https://github.com/kalomaze) - Original RVC colab + Mangio tweaks
*   [Alexlnkp](https://github.com/alexlnkp) - Created a more up-to-date variant of the Mangio-RVC fork
*   [Pony Preservation Project](https://boards.4channel.org/mlp/catalog#s=Pony%20Preservation%20Project) - for their robust TalkNet and So-Vits-SVC notebooks
*   the Colab team - forcing my hand and making me release this notebook early

